# Obtención de la Información.
Una vez descargados los archivos, y colocados en su carpeta correspondiente podemos comenzar con la formateo, es decir, obtener las distintas tablas que vienen dentro de cada hoja.


Ahora, procedemos a llamar el documento recien creado. 

In [7]:
import pandas as pd

sonora = pd.read_excel('..\\medium_data\\book_son.xlsx',sheet_name=None)
municipios = list(sonora.keys())
data = sonora['Aconchi']
data.head()

,Unnamed: 0.1,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,Información básica del municipio de,NaN,NaN,NaN,NaN


Ahora, debemos separar por bloques. Primero eliminamos la primer y segunda columna, son irrelevantes. 

In [8]:
df = data.copy()
df.drop(columns=["Unnamed: 0.1","Unnamed: 0"], inplace=True)
df.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,Información básica del municipio de,NaN,NaN,NaN,NaN


Ahora, debemos separar las distintas tablas. Primero eliminaremos los renglones en blanco. 

In [9]:
df['NA'] = df.isna().sum(axis = 1)
df.head(n = 20)

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,NA
0,NaN,NaN,NaN,NaN,NaN,5
1,NaN,NaN,NaN,NaN,NaN,5
2,NaN,NaN,NaN,NaN,NaN,5
3,NaN,NaN,NaN,NaN,NaN,5
4,Información básica del municipio de,NaN,NaN,NaN,NaN,4
5,Aconchi,NaN,NaN,NaN,NaN,4
6,NaN,NaN,NaN,NaN,NaN,5
7,Composición territorial,Absoluto,NaN,Porcentaje,NaN,2
8,Localidades,25,NaN,-,NaN,2
9,Urbanas,1,NaN,4,NaN,2


Ahora debemos buscar la forma de identificarlas, para ello aprovecharemos que estan separadas por espacios en blanco. 

In [10]:
search_tables = []
for i in range(df.shape[0] - 1):
    if df.iloc[i, -1] == 5:
        if df.iloc[i + 1, -1] < 5:
            search_tables.append(i + 1)
    else:
        if df.iloc[i + 1, -1] == 5:
            search_tables.append(i + 1)

Ahora, guardamos los dataframes. 

In [11]:
new_dfs = []
for k in range(len(search_tables) - 1):
    new_dfs.append(df.iloc[search_tables[k]:search_tables[k + 1], :])
new_dfs.append(df.iloc[search_tables[-1]:, :])

str_dfs = [data.astype(str) for data in new_dfs]
str_rows = [data.shape[0] for data in new_dfs]
select_nas = [(data.iloc[:,0] == 'nan').sum() for data in str_dfs]

ne_dfs = []

for k, data in enumerate(str_dfs):
    if select_nas[k] != str_rows[k]:
        ne_dfs.append(data)

Ahora debemos asegurarnos que está funcione para todos los municipios. Meteremos todo lo realizado anteriormente como una función.

In [12]:
def indicators_tables(Municipio: str):
    data = sonora[Municipio]
    df = data.copy()
    df.drop(columns=["Unnamed: 0.1","Unnamed: 0"], inplace=True)
    df['NA'] = df.isna().sum(axis = 1)
    search_tables = []
    for i in range(df.shape[0] - 1):
        if df.iloc[i, -1] == 5:
            if df.iloc[i + 1, -1] < 5:
                search_tables.append(i + 1)
        else:
            if df.iloc[i + 1, -1] == 5:
                search_tables.append(i + 1)
    new_dfs = []
    for k in range(len(search_tables) - 1):
        new_dfs.append(df.iloc[search_tables[k]:search_tables[k + 1], :])
    new_dfs.append(df.iloc[search_tables[-1]:, :])
    str_dfs = [data.astype(str) for data in new_dfs]
    str_rows = [data.shape[0] for data in new_dfs]
    select_nas = [(data.iloc[:,0] == 'nan').sum() for data in str_dfs]
    ne_dfs = []
    for k, data in enumerate(str_dfs):
        if select_nas[k] != str_rows[k]:
            ne_dfs.append(data)
    return ne_dfs

Ahora crearemos un diccionario que guarde todas las listas de dataframes. 

In [13]:
sonora_indicadores = {municipio: indicators_tables(municipio) for municipio in municipios}



Ahora, obseraremos que sucede con las tablas guardadas por municipio. 

In [14]:
index = 0
x_index = [sonora_indicadores[i][index] for i in municipios]

x_index

[                            Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
 4  Información básica del municipio de        nan        nan        nan   
 5                              Aconchi        nan        nan        nan   
 
   Unnamed: 5 NA  
 4        nan  4  
 5        nan  4  ,
                             Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
 4  Información básica del municipio de        nan        nan        nan   
 5                          Agua Prieta        nan        nan        nan   
 
   Unnamed: 5 NA  
 4        nan  4  
 5        nan  4  ,
                             Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
 4  Información básica del municipio de        nan        nan        nan   
 5                               Alamos        nan        nan        nan   
 
   Unnamed: 5 NA  
 4        nan  4  
 5        nan  4  ,
                             Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
 4  Información básica del municipio de        nan        na

Notamos cierto patrón. Ahora vamos con otro indice. 

In [15]:
index = 1
x_index = [sonora_indicadores[i][index] for i in municipios]
x_index

[                 Unnamed: 1 Unnamed: 2 Unnamed: 3  Unnamed: 4 Unnamed: 5 NA
 7   Composición territorial   Absoluto        nan  Porcentaje        nan  2
 8               Localidades         25        nan           -        nan  2
 9                   Urbanas          1        nan           4        nan  2
 10                  Rurales         24        nan          96        nan  2
 11                     AGEB         14        nan           -        nan  2
 12                  Urbanas         11        nan        78.6        nan  2
 13                  Rurales          3        nan        21.4        nan  2
 14                 Manzanas        223        nan           -        nan  2
 15                  Urbanas        135        nan        60.5        nan  2
 16                  Rurales         88        nan        39.5        nan  2
 17           Fuente: INEGI.        nan        nan         nan        nan  4,
                  Unnamed: 1 Unnamed: 2 Unnamed: 3  Unnamed: 4 Unnamed: 5 N

El patron se repite. Entonces, tenemos la certeza que se aplico de igual forma a todos los municipios. Finalmente guardaremos el diccionario como un .json